In [25]:
import os
import cv2
import numpy as np
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding


In [26]:
# Generate RSA Key Pair
def generate_rsa_keypair():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()

    return private_key, public_key


In [27]:
# Encrypt AES key with RSA
def encrypt_aes_key(aes_key, public_key):
    encrypted_aes_key = public_key.encrypt(
        aes_key,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return encrypted_aes_key


In [28]:
# Decrypt AES key with RSA
def decrypt_aes_key(encrypted_aes_key, private_key):
    decrypted_aes_key = private_key.decrypt(
        encrypted_aes_key,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return decrypted_aes_key


In [29]:
# AES Encryption with Padding
def encrypt_with_aes(image, key):
    cipher = Cipher(algorithms.AES(key), modes.CBC(key[:16]), backend=default_backend())
    encryptor = cipher.encryptor()

    # Flatten the image and apply padding to ensure multiple of 16 bytes
    h, w, c = image.shape
    image_bytes = image.tobytes()
    padder = padding.PKCS7(128).padder()  # 128 bits = 16 bytes
    padded_image = padder.update(image_bytes) + padder.finalize()

    # Encrypt the padded data
    encrypted_image = encryptor.update(padded_image) + encryptor.finalize()

    return encrypted_image, h, w, c


In [30]:
# AES Decryption with Padding Removal
def decrypt_with_aes(encrypted_image, key, h, w, c):
    cipher = Cipher(algorithms.AES(key), modes.CBC(key[:16]), backend=default_backend())
    decryptor = cipher.decryptor()

    # Decrypt the encrypted data
    decrypted_image = decryptor.update(encrypted_image) + decryptor.finalize()

    # Remove padding
    unpadder = padding.PKCS7(128).unpadder()
    decrypted_image = unpadder.update(decrypted_image) + unpadder.finalize()

    # Convert back to numpy array and reshape
    decrypted_image = np.frombuffer(decrypted_image, dtype=np.uint8).reshape(h, w, c)
    
    return decrypted_image


In [31]:
# Main function to load image, encrypt/decrypt with AES and RSA, and save results
def main():
    # Load image
    image_path = input("Enter path of the image to encrypt: ")
    image = cv2.imread(image_path)
    
    if image is None:
        print(f"Error: Unable to read image from {image_path}")
        return

    # Generate AES key and RSA keys
    aes_key = os.urandom(32)  # AES-256 key
    private_key, public_key = generate_rsa_keypair()

    # Encrypt AES key with RSA
    print("[INFO] Encrypting AES Key using RSA...")
    encrypted_aes_key = encrypt_aes_key(aes_key, public_key)

    # Encrypt image with AES
    print("[INFO] Encrypting Image with AES...")
    encrypted_image, h, w, c = encrypt_with_aes(image, aes_key)
    # Save the encrypted data (not the reshaped image)
    with open('aes_encrypted_image.bin', 'wb') as f:
        f.write(encrypted_image)
    print("[INFO] Encrypted image saved!")

    # Decrypt AES key using RSA
    print("[INFO] Decrypting AES Key using RSA...")
    decrypted_aes_key = decrypt_aes_key(encrypted_aes_key, private_key)

    # Decrypt image with AES
    print("[INFO] Decrypting Image with AES...")
    decrypted_image = decrypt_with_aes(encrypted_image, decrypted_aes_key, h, w, c)
    cv2.imwrite('aes_decrypted_image.jpg', decrypted_image)
    print("[INFO] Decrypted image saved!")

    print("[SUCCESS] AES Image Encryption & Decryption Completed!")

if __name__ == "__main__":
    main()


[INFO] Encrypting AES Key using RSA...
[INFO] Encrypting Image with AES...
[INFO] Encrypted image saved!
[INFO] Decrypting AES Key using RSA...
[INFO] Decrypting Image with AES...
[INFO] Decrypted image saved!
[SUCCESS] AES Image Encryption & Decryption Completed!
